In [1]:
# -*- coding: utf-8 -*-                                                                        
import numpy as np
import itertools as it
import onnx
from onnx import helper, checker
from onnx import TensorProto
import os, psutil
from onnx import numpy_helper
from onnx import AttributeProto, TensorProto, GraphProto

# import autodice functions .

from code_generator import CppFile
from cpp_generator import *
from onnx_split import *
from data_json import *
from interface import *

In [2]:
 
# CNN Mapping:  
# {'nx01_arm012345': ['conv1_1', 'conv1_2', 'norm1_1', 
# 'pool1_1', 'conv2_1', 'conv2_2', 'norm2_1', 'pool2_1', 'conv3_1', 
# 'conv3_2', 'conv4_1', 'conv4_2', 'conv5_1', 'conv5_2', 'pool5_1', 
# 'OC2_DUMMY_0', 'fc6_1', 'fc6_2'], 
# 'nx01_gpu': ['fc6_3', 'fc7_1', 'fc7_2', 'fc7_3', 'fc8_1', 'prob_1']}

origin_model = "bvlcalexnet-9.onnx"
input_model = format_onnx(origin_model)  # name all layers according to their output tensor name
model =  onnx.load(input_model) 
model_len = len(model.graph.node)

resourceid = { 1:'lenovo_cpu0', 2:'lenovo_cpu1', 3:'lenovo_cpu2'}
platforms = ['lenovo']
cnn_map = {
        'lenovo_cpu0': [ 'conv1_2', 'norm1_1', 
            'pool1_1', 'conv2_1', 'conv2_2', 'norm2_1', 'pool2_1', 'conv3_1', 
            'conv3_2', 'conv4_1', 'conv4_2', 'conv5_1', 'conv5_2', 'pool5_1', 
            'OC2_DUMMY_0', 'fc6_1', 'fc6_2'], 
         'lenovo_cpu1': ['fc6_3', 'fc7_1', 'fc7_2', 'fc7_3', 'fc8_1', 'prob_1'],
         'lenovo_cpu2':['conv1_1']}


split_axis = 2
split_ranks = [0,1]
split_points = [6,9300]
splitnode_name = 'conv2_1'
model =  onnx.load(input_model) 
new_model = horizontal_split(model, splitnode_name, split_ranks, split_axis, split_points)
onnx.save(new_model, "lip.onnx")


 MODEL has total 24 layers.
Split Convolution Layer (LIP--Width)  conv2_1


In [7]:
model =  onnx.load(origin_model) 
model = onnx.shape_inference.infer_shapes(model)
graph = model.graph
input_map = generate_node_dict(graph.input)
output_map = generate_node_dict(graph.output)
initializer_map = generate_node_dict(graph.initializer)
value_map = generate_node_dict(graph.value_info)
node_map = generate_node_dict(graph.node)

In [8]:
value_map.keys()

dict_keys(['conv1_1', 'conv1_2', 'norm1_1', 'pool1_1', 'conv2_1', 'conv2_2', 'norm2_1', 'pool2_1', 'conv3_1', 'conv3_2', 'conv4_1', 'conv4_2', 'conv5_1', 'conv5_2', 'pool5_1', 'OC2_DUMMY_0', 'fc6_1', 'fc6_2', 'fc6_3', 'fc7_1', 'fc7_2', 'fc7_3', 'fc8_1'])

In [21]:
value_map['conv2_1'].type.tensor_type.shape.dim

[dim_value: 1
, dim_value: 256
, dim_value: 26
, dim_value: 26
]

In [3]:
split_starts = np.zeros(np.shape(split_ranks), dtype=int)
split_ends = np.zeros(np.shape(split_ranks),  dtype=int)

graph = model.graph
for n in graph.node:
    if n.name == '':
        n.name = str(n.output[0])

nodeIdx = find_node_index(graph, splitnode_name)

input_map = generate_node_dict(graph.input)
output_map = generate_node_dict(graph.output)
initializer_map = generate_node_dict(graph.initializer)
value_map = generate_node_dict(graph.value_info)
node_map = generate_node_dict(graph.node)

node_inputs = [n for n in list(node_map[splitnode_name].input) if n not in initializer_map]

###############
### Split Input
###############
split_output_names = []
for i in range(len(split_ranks)):
    split_output_names.append(splitnode_name+'_hsplit_'+str(i))

gemm_attributes = {'alpha':1.0, 'beta':1.0, 'transA':0, 'transB':1}
conv_attributes = {'auto_pad':[],'dilations':[],'group':1,'kernel_shape':[],'pads':[],'strides':[]}
fc_attributes = {}
node_attribute_names = []
for i in range(len(node_map[splitnode_name].attribute)):
    node_attribute_names.append(node_map[splitnode_name].attribute[i].name)

# Get attributes of Conv layer, pads, kernel, stride, etc.

if node_map[splitnode_name].op_type == 'Conv':
    for i in range(len(node_map[splitnode_name].attribute)):
        attribute_name = node_attribute_names[i]
        if node_map[splitnode_name].attribute[i].ints:
            conv_attributes[attribute_name] = node_map[splitnode_name].attribute[i].ints
        else:
            conv_attributes[attribute_name] = node_map[splitnode_name].attribute[i].i


if node_map[splitnode_name].op_type == 'Gemm':   
    for i in range(len(node_map[splitnode_name].attribute)):
        attribute_name = node_attribute_names[i] 
        if node_map[splitnode_name].attribute[i].ints:                      
            gemm_attributes[attribute_name] = node_map[splitnode_name].attribute[i].ints
        else:                                    
            gemm_attributes[attribute_name] = node_map[splitnode_name].attribute[i].i

weight_names = [name for name in list(node_map[splitnode_name].input) if name in initializer_map]
for name in weight_names:
    if len(initializer_map[name].dims) == 4: # Cout * Cin * Kh * Kw.
        w = onnx.numpy_helper.to_array(initializer_map[name]) 
    if len(initializer_map[name].dims) == 2: # Cout * Cin
        w = onnx.numpy_helper.to_array(initializer_map[name]) 
    if len(initializer_map[name].dims) == 1: # Cout.
        b = onnx.numpy_helper.to_array(initializer_map[name])  

KeyError: 'conv2_1'

In [4]:
np.shape(w)[1]

48

In [8]:
import numpy as np
split_ranks = [0,1]

split_ranks = np.array(split_ranks, dtype=int)
split_ranks = np.repeat(split_ranks, 2)
split_ranks

array([0, 0, 1, 1])

In [10]:

def find_node_index(graph, node_name):
    idx = 0
    for n in graph.node:
        if n.name == node_name:
            return idx
        idx = idx +1
    return -1

# The protobuf definition can be found here:
# https://github.com/onnx/onnx/blob/main/onnx/onnx.proto
split_axis = 2
split_ranks = [0,1]
split_points = [3,3000]
split_starts = np.zeros(np.shape(split_ranks), dtype=int)
split_ends = np.zeros(np.shape(split_ranks),  dtype=int)

# split_ranks = (0,1)
# split_points = (1,3000)
# split_starts = (0,1)
# split_ends = (1,3001)
split_output_names = []

splitnode_name = 'conv4_1'
#splitnode_name = 'conv1_1'
# model = onnx.shape_inference.infer_shapes(model)

graph = model.graph 
#Generate a name for all node if they have none.

for n in graph.node:
    if n.name == '':
        n.name = str(n.output[0])

nodeIdx = find_node_index(graph, splitnode_name)
         
        
input_tensors = getInputlayers(input_model)
input_map = generate_node_dict(graph.input)
output_map = generate_node_dict(graph.output)
initializer_map = generate_node_dict(graph.initializer)
value_map = generate_node_dict(graph.value_info)
node_map = generate_node_dict(graph.node)

node_inputs = [n for n in list(node_map[splitnode_name].input) if n not in initializer_map]


for i in range(len(split_ranks)):
    split_output_names.append(splitnode_name+'_hsplit_'+str(i))

    
conv_attributes = {'auto_pad':[],'dilations':[],'group':[1],'kernel_shape':[],'pads':[],'strides':[]}
fc_attributes = {}
if node_map[splitnode_name].op_type == 'Conv':
    node_attribute_names = []
    for i in range(len(node_map[splitnode_name].attribute)):
        node_attribute_names.append(node_map[splitnode_name].attribute[i].name)
    for i in range(len(node_map[splitnode_name].attribute)):
        attribute_name = node_attribute_names[i]
        if node_map[splitnode_name].attribute[i].ints:
            conv_attributes[attribute_name] = node_map[splitnode_name].attribute[i].ints
        else:
            conv_attributes[attribute_name] = node_map[splitnode_name].attribute[i].i

if split_axis == -1: # do not split for the input.
    _offset = 0
    for i in range(len(split_ranks)):
        split_starts[i] = 0
        split_ends[i] = 0
        _offset = split_ends[i]
        
if split_axis == 0: # channel
    _offset = 0
    
    for i in range(len(split_ranks)):
        split_starts[i] = _offset
        split_ends[i] = _offset + split_points[i]
        _offset = split_ends[i]
        
if split_axis == 1: # height
    _offset = -conv_attributes['pads'][0] 
    for i in range(len(split_ranks)):
        split_starts[i] = max(_offset, 0)
        split_ends[i] = _offset + (split_points[i]-1) * conv_attributes['strides'][0] 
        _offset = split_ends[i] + conv_attributes['strides'][0]
        split_ends[i] = max(split_ends[i]+ conv_attributes['kernel_shape'][0], 0)
        
if split_axis == 2: # width.
    _offset = -conv_attributes['pads'][0] 
    for i in range(len(split_ranks)):
        split_starts[i] = max(_offset, 0)
        split_ends[i] = _offset + (split_points[i]-1) * conv_attributes['strides'][0] 
        _offset = split_ends[i] + conv_attributes['strides'][0] 
        split_ends[i] = max(split_ends[i]+ conv_attributes['kernel_shape'][0], 0)

print ("split_starts: ",split_starts)
print ("split_ends: ", split_ends)    
    
if split_axis<0:
    conv_hsplit = onnx.helper.make_node(
        name=splitnode_name + "_hsplit",  # Name is optional.
        op_type = "NSplit",
        inputs = node_inputs, # inputs
        outputs=split_output_names, # outputs    
    )    
else:
# Create a node (NodeProto)
    conv_hsplit = onnx.helper.make_node(
        name=splitnode_name + "_hsplit",  # Name is optional.
        op_type = "HSplit",
         inputs = node_inputs, # inputs
        outputs=split_output_names, # outputs    
        axis=split_axis,
        starts=split_starts,
        ends=split_ends,
        sranks=split_ranks,
        spoints=split_points,
    )

graph.node.insert(nodeIdx,conv_hsplit)
nodeIdx = nodeIdx + 1
 

split_starts:  [0 2]
split_ends:  [   4 3004]


In [11]:
# graph_node_names = [n for n in list(node_map) if n in sub_node_names]
# input_node_names = [n for n in list(input_map) if n in input_names]
# output_node_names = [n for n in list(output_map) if n in output_names]
splitnode_weights = {}
splitnode_bias = {}

#if node_map[splitnode_name].op_type == 'Conv':
    if split_axis == -1:
        print ("Split Layer (LOP)")
        conv_output_names = []
        weight_names = [name for name in list(node_map[splitnode_name].input) if name in initializer_map]
        for name in weight_names:
            if node_map[splitnode_name].op_type == 'Conv':
                if len(initializer_map[name].dims) == 4: ### Cout * Cin * Kh * Kw
                    # should be weights not bias.
                    print ("Split Convolution weights Channel")

                    w = onnx.numpy_helper.to_array(initializer_map[name]) 
                    _offset = 0
                    for i in range(len(split_ranks)): 
                        # split into n ranks. n = len(conv1_1_split_ranks)
                        new_splitnode_weight_name = splitnode_name + '_w_' + str(i)
                        splitnode_weights[new_splitnode_weight_name] = w[_offset: split_points[i]+_offset,:,:,:]
                        _offset = _offset + split_points[i]
            
            if node_map[splitnode_name].op_type == 'Gemm':
                if len(initializer_map[name].dims) == 2: # Cout * Cin
                    # should be weights not bias. 
                    print ("Split Fully-Connect weights Channel")

                    w = onnx.numpy_helper.to_array(initializer_map[name]) 
                    _offset = 0
                    for i in range(len(split_ranks)): 
                        # split into n ranks. n = len(conv1_1_split_ranks)
                        new_splitnode_weight_name = splitnode_name + '_w_' + str(i)
                        splitnode_weights[new_splitnode_weight_name] = w[_offset: split_points[i]+_offset,:]
                        _offset = _offset + split_points[i]                                
            if len(initializer_map[name].dims) == 1:
                b = onnx.numpy_helper.to_array(initializer_map[name]) 
                _offset = 0
                for i in range(len(split_ranks)):                     
                    new_splitnode_bias_name = splitnode_name + '_b_'  + str(i)
                    splitnode_bias[new_splitnode_bias_name] = b[_offset: split_points[i]+_offset]
                    _offset = _offset + split_points[i]


        #conv_attributes
        
        for i in range(len(split_ranks)):            
            conv1_output_node_name = splitnode_name + '_csplit_' +str(i)
            conv_output_names.append(conv1_output_node_name)
            # default the first rank(node) with bias.
            W_initializer_tensor_name = splitnode_name + '_w_' + str(i)
            W_initializer_tensor = create_initializer_tensor(
                name=W_initializer_tensor_name,
                tensor_array=splitnode_weights[W_initializer_tensor_name],
                data_type=onnx.TensorProto.FLOAT)            
            
            B_initializer_tensor_name = splitnode_name + '_b_' + str(i)
                
            B_initializer_tensor = create_initializer_tensor(
                    name=B_initializer_tensor_name,
                    tensor_array=splitnode_bias[B_initializer_tensor_name],
                    data_type=onnx.TensorProto.FLOAT)
                        
            graph.initializer.append(W_initializer_tensor)
            graph.initializer.append(B_initializer_tensor)
            if node_map[splitnode_name].op_type == 'Conv':
                conv1_node = onnx.helper.make_node(
                                name=conv1_output_node_name,  # Name is optional.
                                op_type="Conv",
                # Must follow the order of input and output definitions.
                # https://github.com/onnx/onnx/blob/rel-1.9.0/docs/Operators.md#inputs-2---3
                    inputs=[
                            splitnode_name+'_hsplit_'+str(i), W_initializer_tensor_name,
                            B_initializer_tensor_name],
                    outputs=[conv1_output_node_name],
                # The following arguments are attributes.
                    auto_pad =  conv_attributes['auto_pad'],
                    dilations = conv_attributes['dilations'],
                    group = conv_attributes['group'],
                    kernel_shape = conv_attributes['kernel_shape'],
                    pads = conv_attributes['pads'],
                    strides = conv_attributes['strides']
                )
            if node_map[splitnode_name].op_type == 'Gemm':
                conv1_node = onnx.helper.make_node(
                                name=conv1_output_node_name,  # Name is optional.
                                op_type="Gemm",
                # Must follow the order of input and output definitions.
                # https://github.com/onnx/onnx/blob/rel-1.9.0/docs/Operators.md#inputs-2---3
                    inputs=[
                            splitnode_name+'_hsplit_'+str(i), W_initializer_tensor_name,
                            B_initializer_tensor_name],
                    outputs=[conv1_output_node_name],
                # The following arguments are attributes.
                )
            graph.node.insert(nodeIdx, conv1_node)
            nodeIdx = nodeIdx + 1
        # Create a node (NodeProto)
        conv_hsum = onnx.helper.make_node(
            name=splitnode_name + "_oconcat",  # Name is optional.
            op_type = "Concat",
            inputs = conv_output_names, # inputs
            outputs= node_map[splitnode_name].output, # outputs    
            axis=0
        )
        for name in weight_names:
            graph.input.remove(input_map[name])
            graph.initializer.remove(initializer_map[name])
        graph.node.insert(nodeIdx,conv_hsum)
        nodeIdx = nodeIdx + 1
        
    if split_axis == 0:  
        print ("Split Convolution Layer Channel")
        conv_output_names = []
        weight_names = [name for name in list(node_map[splitnode_name].input) if name in initializer_map]
        for name in weight_names:
            if len(initializer_map[name].dims) == 4:
                # should be weights not bias.
                print ("Split Convolution weights Channel")

                w = onnx.numpy_helper.to_array(initializer_map[name]) 
                _offset = 0
                for i in range(len(split_ranks)): 
                    # split into n ranks. n = len(conv1_1_split_ranks)
                    new_splitnode_weight_name = splitnode_name + '_w_' + str(i)
                    splitnode_weights[new_splitnode_weight_name] = w[:,_offset: split_points[i]+_offset,:,:]
                    _offset = _offset + split_points[i]
                                
            if len(initializer_map[name].dims) == 1:
                b = onnx.numpy_helper.to_array(initializer_map[name]) 
                new_splitnode_bias_name = splitnode_name + '_b_0' 
                splitnode_bias[new_splitnode_bias_name] = b


        #conv_attributes   
        for i in range(len(split_ranks)):            
            conv1_output_node_name = splitnode_name + '_splito_' +str(i)
            conv_output_names.append(conv1_output_node_name)
            # default the first rank(node) with bias.
            W_initializer_tensor_name = splitnode_name + '_w_' + str(i)
            W_initializer_tensor = create_initializer_tensor(
                name=W_initializer_tensor_name,
                tensor_array=splitnode_weights[W_initializer_tensor_name],
                data_type=onnx.TensorProto.FLOAT)            
            
            B_initializer_tensor_name = splitnode_name + '_b_' + str(i)
            if i==0:    
                B_initializer_tensor = create_initializer_tensor(
                    name=B_initializer_tensor_name,
                    tensor_array=splitnode_bias[B_initializer_tensor_name],
                    data_type=onnx.TensorProto.FLOAT)
            else:
                B_initializer_tensor = create_initializer_tensor(
                    name=B_initializer_tensor_name,
                    tensor_array=np.array([],dtype=float),
                    data_type=onnx.TensorProto.FLOAT)
                        
            graph.initializer.append(W_initializer_tensor)
            graph.initializer.append(B_initializer_tensor)
    
            conv1_node = onnx.helper.make_node(
                            name=conv1_output_node_name,  # Name is optional.
                            op_type="Conv",
            # Must follow the order of input and output definitions.
            # https://github.com/onnx/onnx/blob/rel-1.9.0/docs/Operators.md#inputs-2---3
                inputs=[
                        splitnode_name+'_hsplit_'+str(i), W_initializer_tensor_name,
                        B_initializer_tensor_name],
                outputs=[conv1_output_node_name],
            # The following arguments are attributes.
                auto_pad =  conv_attributes['auto_pad'],
                dilations = conv_attributes['dilations'],
                group = conv_attributes['group'],
                kernel_shape = conv_attributes['kernel_shape'],
                pads = conv_attributes['pads'],
                strides = conv_attributes['strides']
            )
            
            graph.node.insert(nodeIdx, conv1_node)
            nodeIdx = nodeIdx + 1
        # Create a node (NodeProto)
        conv_hsum = onnx.helper.make_node(
            name=splitnode_name + "_hsum",  # Name is optional.
            op_type = "HSum",
            inputs = conv_output_names, # inputs
            outputs= node_map[splitnode_name].output, # outputs    
            axis=split_axis,
            sranks=split_ranks,
        )
        for name in weight_names:
            graph.input.remove(input_map[name])
            graph.initializer.remove(initializer_map[name])
        graph.node.insert(nodeIdx,conv_hsum)
        nodeIdx = nodeIdx + 1
 
            
                # must be bias.
    if split_axis == 1:  
        print ("Split Convolution Layer Height")    
        conv_output_names = []
        weight_names = [name for name in list(node_map[splitnode_name].input) if name in initializer_map]
        for name in weight_names:
            if len(initializer_map[name].dims) == 4:
                # should be weights not bias.
                w = onnx.numpy_helper.to_array(initializer_map[name]) 
                for i in range(len(split_ranks)): 
                    # split into n ranks. n = len(conv1_1_split_ranks)
                    new_splitnode_weight_name = splitnode_name + '_w_' + str(i)
                    splitnode_weights[new_splitnode_weight_name] = w
                    
                                
            if len(initializer_map[name].dims) == 1:
                b = onnx.numpy_helper.to_array(initializer_map[name]) 
                _offset = 0                
                for i in range(len(split_ranks)): 
                    new_splitnode_bias_name = splitnode_name + '_b_' + str(i)
                    splitnode_bias[new_splitnode_bias_name] = b


        #conv_attributes   
        for i in range(len(split_ranks)):            
            conv1_output_node_name = splitnode_name + '_splith_' +str(i)
            conv_output_names.append(conv1_output_node_name)
            # default the first rank(node) with bias.
            W_initializer_tensor_name = splitnode_name + '_w_' + str(i)
            W_initializer_tensor = create_initializer_tensor(
                name=W_initializer_tensor_name,
                tensor_array=splitnode_weights[W_initializer_tensor_name],
                data_type=onnx.TensorProto.FLOAT)            
            
            B_initializer_tensor_name = splitnode_name + '_b_' + str(i)
            B_initializer_tensor = create_initializer_tensor(
                    name=B_initializer_tensor_name,
                    tensor_array=splitnode_bias[B_initializer_tensor_name],
                    data_type=onnx.TensorProto.FLOAT)
                        
            graph.initializer.append(W_initializer_tensor)
            graph.initializer.append(B_initializer_tensor)
    
            conv1_node = onnx.helper.make_node(
                            name=conv1_output_node_name,  # Name is optional.
                            op_type="Conv",
            # Must follow the order of input and output definitions.
            # https://github.com/onnx/onnx/blob/rel-1.9.0/docs/Operators.md#inputs-2---3
                inputs=[
                        splitnode_name+'_hsplit_'+str(i), W_initializer_tensor_name,
                        B_initializer_tensor_name],
                outputs=[conv1_output_node_name],
            # The following arguments are attributes.
                auto_pad =  conv_attributes['auto_pad'],
                dilations = conv_attributes['dilations'],
                group = conv_attributes['group'],
                kernel_shape = conv_attributes['kernel_shape'],
                pads = conv_attributes['pads'],
                strides = conv_attributes['strides']
            )
            
            graph.node.insert(nodeIdx, conv1_node)
            nodeIdx = nodeIdx + 1
        # Create a node (NodeProto)
        conv_hsum = onnx.helper.make_node(
            name=splitnode_name + "_hconcat",  # Name is optional.
            op_type = "Concat",
            inputs = conv_output_names, # inputs
            outputs= node_map[splitnode_name].output, # outputs    
            axis=1,
         )
        for name in weight_names:
            graph.input.remove(input_map[name])
            graph.initializer.remove(initializer_map[name])
        graph.node.insert(nodeIdx,conv_hsum)
        nodeIdx = nodeIdx + 1
        
        
    if split_axis == 2:  
        print ("Split Convolution Layer Width")    
        conv_output_names = []
        weight_names = [name for name in list(node_map[splitnode_name].input) if name in initializer_map]
        for name in weight_names:
            if len(initializer_map[name].dims) == 4:
                # should be weights not bias.
                print ("Split Convolution weights Channel")
                w = onnx.numpy_helper.to_array(initializer_map[name]) 
                for i in range(len(split_ranks)): 
                    # split into n ranks. n = len(conv1_1_split_ranks)
                    new_splitnode_weight_name = splitnode_name + '_w_' + str(i)
                    splitnode_weights[new_splitnode_weight_name] = w
                    
                                
            if len(initializer_map[name].dims) == 1:                                
                b = onnx.numpy_helper.to_array(initializer_map[name]) 
                for i in range(len(split_ranks)): 
                    b = onnx.numpy_helper.to_array(initializer_map[name]) 
                    new_splitnode_bias_name = splitnode_name + '_b_' + str(i)
                    splitnode_bias[new_splitnode_bias_name] = b


        #conv_attributes   
        for i in range(len(split_ranks)):            
            conv1_output_node_name = splitnode_name + '_splitw_' +str(i)
            conv_output_names.append(conv1_output_node_name)
            # default the first rank(node) with bias.
            W_initializer_tensor_name = splitnode_name + '_w_' + str(i)
            W_initializer_tensor = create_initializer_tensor(
                name=W_initializer_tensor_name,
                tensor_array=splitnode_weights[W_initializer_tensor_name],
                data_type=onnx.TensorProto.FLOAT)            
            
            B_initializer_tensor_name = splitnode_name + '_b_' + str(i)
            B_initializer_tensor = create_initializer_tensor(
                    name=B_initializer_tensor_name,
                    tensor_array=splitnode_bias[B_initializer_tensor_name],
                    data_type=onnx.TensorProto.FLOAT)
                        
            graph.initializer.append(W_initializer_tensor)
            graph.initializer.append(B_initializer_tensor)
    
            conv1_node = onnx.helper.make_node(
                            name=conv1_output_node_name,  # Name is optional.
                            op_type="Conv",
            # Must follow the order of input and output definitions.
            # https://github.com/onnx/onnx/blob/rel-1.9.0/docs/Operators.md#inputs-2---3
                inputs=[
                        splitnode_name+'_hsplit_'+str(i), W_initializer_tensor_name,
                        B_initializer_tensor_name],
                outputs=[conv1_output_node_name],
            # The following arguments are attributes.
                auto_pad =  conv_attributes['auto_pad'],
                dilations = conv_attributes['dilations'],
                group = conv_attributes['group'],
                kernel_shape = conv_attributes['kernel_shape'],
                pads = conv_attributes['pads'],
                strides = conv_attributes['strides']
            )
            
            graph.node.insert(nodeIdx, conv1_node)
            nodeIdx = nodeIdx + 1
        # Create a node (NodeProto)
        conv_hsum = onnx.helper.make_node(
            name=splitnode_name + "_wconcat",  # Name is optional.
            op_type = "Concat",
            inputs = conv_output_names, # inputs
            outputs= node_map[splitnode_name].output, # outputs    
            axis=2,
         )
        for name in weight_names:
            graph.input.remove(input_map[name])
            graph.initializer.remove(initializer_map[name])
        graph.node.insert(nodeIdx,conv_hsum)
        nodeIdx = nodeIdx + 1
        
    nodeIdx = find_node_index(graph, splitnode_name)
    graph.node.remove(graph.node[nodeIdx])    
    onnx.save(model, "modified.onnx")

Split Convolution Layer Height


In [8]:
for k, v in splitnode_weights.items():
    print (k, np.shape(v)) 

conv4_1_w_0 (384, 1, 3, 3)
conv4_1_w_1 (384, 191, 3, 3)


In [14]:
# call interface function to generate onnx models
# Check whether the specified path exists or not
output_dirs= './models'
if not os.path.exists(output_dirs):
  # Create a new directory because it does not exist 
  os.makedirs(output_dirs)
  print("The output directory %s is created!" % (output_dirs))

start_time = time.time()
InputSpecs = Interface(model=input_model, mappings=cnn_map, platforms=platforms)
print ("Front End time: %f (s)"%(time.time() - start_time))
output_dirs = 'models'



The output directory ./models is created!
Total GPU: 1, Total CPU: 6
Consistency Check Pass.
Devices: 1
Generate  2  Sub-Models.
Front End time: 5.287814 (s)


In [15]:
# Generate cpp code 
GenerateCode = EngineCode(
    CppName = "./models/multinode",
    Platforms = InputSpecs.platforms,
    NodesList = InputSpecs.nodes,
    ComputingNodes = InputSpecs.computingnodes,
    ValueInfos = InputSpecs.value_map,
    Inputs = InputSpecs.inputs,
    Outputs = InputSpecs.outputs,
    Benchmark = False)
